In [6]:
import numpy as np
import DnnLib

# Cargar test set
data_test = np.load("mnist_test.npz")
images_test = data_test["images"].astype(np.float64)
labels_test = data_test["labels"]

# Normalizar y vectorizar
X_test = images_test.reshape(len(images_test), -1) / 255.0

# One-hot encoding
def to_categorical(y, num_classes):
    return np.eye(num_classes)[y]

y_test = to_categorical(labels_test, 10)

# Para demo (sin mnist_train.npz) usamos parte de test como train
X_train, y_train = X_test[:8000], y_test[:8000]
X_val, y_val = X_test[8000:], y_test[8000:]


In [7]:
layers = [
    DnnLib.DenseLayer(784, 128, DnnLib.ActivationType.RELU),
    DnnLib.DenseLayer(128, 64, DnnLib.ActivationType.RELU),
    DnnLib.DenseLayer(64, 10, DnnLib.ActivationType.SOFTMAX)
]

In [8]:
optimizer = DnnLib.Adam(learning_rate=0.001)


In [9]:
def train_minibatch(layers, optimizer, X, y, batch_size=64, epochs=10):
    n_samples = X.shape[0]
    for epoch in range(epochs):
        indices = np.random.permutation(n_samples)
        X_shuff, y_shuff = X[indices], y[indices]
        epoch_loss, n_batches = 0.0, 0
        
        for i in range(0, n_samples, batch_size):
            X_batch = X_shuff[i:i+batch_size]
            y_batch = y_shuff[i:i+batch_size]

            # Forward
            activation = X_batch
            for layer in layers:
                activation = layer.forward(activation)
            output = activation

            # Loss + grad
            loss = DnnLib.cross_entropy(output, y_batch)
            grad = DnnLib.cross_entropy_gradient(output, y_batch)

            # Backward
            for layer in reversed(layers):
                grad = layer.backward(grad)
                optimizer.update(layer)

            epoch_loss += loss
            n_batches += 1

        print(f"Epoch {epoch}, Loss: {epoch_loss/n_batches:.4f}")


In [10]:
train_minibatch(layers, optimizer, X_train, y_train, batch_size=64, epochs=10)

Epoch 0, Loss: 0.7233
Epoch 1, Loss: 0.2792
Epoch 2, Loss: 0.2017
Epoch 3, Loss: 0.1518
Epoch 4, Loss: 0.1157
Epoch 5, Loss: 0.0862
Epoch 6, Loss: 0.0654
Epoch 7, Loss: 0.0515
Epoch 8, Loss: 0.0376
Epoch 9, Loss: 0.0287


In [11]:
# Forward en todo test (sin dropout)
preds = X_test
for layer in layers:
    preds = layer.forward(preds)

pred_classes = np.argmax(preds, axis=1)
accuracy = np.mean(pred_classes == labels_test)
print("Accuracy en test:", accuracy)


Accuracy en test: 0.9904


In [ ]:
import json

def save_model(layers, filename="model.json"):
    model_data = []
    for layer in layers:
        if isinstance(layer, DnnLib.DenseLayer):
            model_data.append({
                "type": "DenseLayer",
                "input_dim": layer.weights.shape[1],
                "output_dim": layer.weights.shape[0],
                "activation": layer.activation_type.name,
                "weights": layer.weights.tolist(),
                "bias": layer.bias.tolist()
            })
    with open(filename, "w") as f:
        json.dump(model_data, f)
    print(f"Modelo guardado en {filename}")


def load_model(filename="model.json"):
    with open(filename, "r") as f:
        model_data = json.load(f)

    layers = []
    for layer_info in model_data:
        if layer_info["type"] == "DenseLayer":
            layer = DnnLib.DenseLayer(
                layer_info["input_dim"],
                layer_info["output_dim"],
                getattr(DnnLib.ActivationType, layer_info["activation"])
            )
            layer.weights[:] = np.array(layer_info["weights"], dtype=np.float64)
            layer.bias[:] = np.array(layer_info["bias"], dtype=np.float64)
            layers.append(layer)
    print(f"Modelo cargado desde {filename}")
    return layers

# Después de entrenar
#save_model(layers, "mnist_model.json")

# Más tarde (o en otro script)
#loaded_layers = load_model("mnist_model.json")

# Hacer predicciones con el modelo cargado
#preds = X_test
#for layer in loaded_layers:
#   preds = layer.forward(preds)

#acc = np.mean(np.argmax(preds, axis=1) == labels_test)
#print("Accuracy del modelo cargado:", acc)
